In [2]:
# hard negative 만들기
from utils.utils import *
from tqdm import tqdm
from transformers import AutoTokenizer
import argparse
from sklearn.model_selection import train_test_split
import json

In [6]:
a=np.array_split(x,4)

In [10]:
a

[[1], [2], [3], [4]]

In [4]:
 a = np.array_split(data, args.num_cores)

NameError: name 'data' is not defined

In [2]:
parser = argparse.ArgumentParser()
# test name
parser.add_argument('--contexts_path', type=str, default='../../data/dpr/docs.jsonl')
parser.add_argument('--data_path', type=str, default='../../data/dpr/dev.jsonl')

_StoreAction(option_strings=['--data_path'], dest='data_path', nargs=None, const=None, default='../../data/dpr/dev.jsonl', type=<class 'str'>, choices=None, help=None, metavar=None)

In [3]:
args,_ = parser.parse_known_args()

In [4]:
contexts = load_jsonl(args.contexts_path)

FileNotFoundError: [Errno 2] No such file or directory: '../../data/dpr/docs.jsonl'

In [4]:
data = load_jsonl(args.data_path)

121746it [00:01, 65005.53it/s]


In [6]:
doc_ids = [i['doc_id'] for i in contexts]
for i,j in zip(doc_ids, range(len(contexts))):
    assert i == j

In [5]:
data = [i['positive_ctxs']*100 for i in data]

In [6]:
import time
from itertools import chain
now = time.time()
flatten_list = list(chain.from_iterable(data))
print(time.time()-now)

0.25499844551086426


In [ ]:
import torch

In [ ]:
a = torch.randn((32,2))

In [ ]:
a[:

In [ ]:
import time
#from itertools import chain
now = time.time()
data = sum(data,[])
#flatten_list = list(chain.from_iterable(data))
print(time.time()-now)

In [7]:
n = 100
total_contexts = [i['tc'] for i in contexts]
corpus = list(map(lambda i : i.split(), total_contexts))
bm25 = BM25(corpus)
for i in [data]:
    for j in tqdm(i, desc='attach'):
        retrieved_result = bm25.get_top_n(j['question'].split(), corpus, n)
        j['retrieved_ctxs_ids'] = [corpus.index(k) for k in retrieved_result]

100%|█████████████████████████████████████████████████████████████████████| 892297/892297 [00:00<00:00, 1113753.19it/s]


Dropping 620 terms


attach: 100%|█████████████████████████████████████████████████████████████████████| 9782/9782 [01:28<00:00, 111.04it/s]


In [11]:
doc_id_2_tc = {i['doc_id']:i['tc'] for i in contexts}

In [12]:
for i in data:
    i['retrieved_ctxs'] = [doc_id_2_tc[j] for j in i['retrieved_ctxs_ids']]

# test_data score 기록

In [15]:
# hit@1, 5, 20, 100
scores = []
for k in range(1, 101):
    score_k = []
    for i in tqdm(data):
        score_k.append(i['positive_ctxs_ids'][0] in i['retrieved_ctxs_ids'][:k])
    scores.append(score_k)

100%|██████████████████████████████████████████████████████████████████████████| 9782/9782 [00:00<00:00, 404462.56it/s]


In [17]:
for i,j in zip(scores,range(1,101)):
    print(j)
    print(sum(i)/len(i))
    print()

1
0.5528521774688203

2
0.6215497853199755

3
0.6557963606624412

4
0.676753220200368

5
0.6891228787569005

6
0.6999591085667553

7
0.7096708239623799

8
0.7173379676957677

9
0.7236761398487017

10
0.7299120834185239

11
0.7344101410754447

12
0.739930484563484

13
0.744939685135964

14
0.7496421999591085

15
0.753731343283582

16
0.7565937436107135

17
0.7592516867716214

18
0.7622163156818647

19
0.7641586587609895

20
0.7663054590063382

21
0.768963402167246

22
0.7711102024125945

23
0.7730525454917195

24
0.7747904314046208

25
0.7764260887344101

26
0.7779595174810877

27
0.7794929462277653

28
0.7812308321406666

29
0.7829687180535678

30
0.7843999182171335

31
0.7853199754651401

32
0.7863422612962584

33
0.7874667757104886

34
0.7882846043753834

35
0.7900224902882846

36
0.7908403189531793

37
0.7915559190349621

38
0.7923737476998569

39
0.7938049478634226

40
0.7949294622776528

41
0.7963606624412186

42
0.7978940911878961

43
0.7985074626865671

44
0.7993252913514619

45

In [20]:
save_jsonl('../../data/dpr',data,'dev_w_retrieved')

In [21]:
data = load_jsonl('../../data/dpr/dev_w_retrieved.jsonl')

9782it [00:05, 1679.64it/s]
